In [3]:
import os
file_name='review_text.txt'
current_dir= os.getcwd()

In [4]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np


In [5]:
max_len = 0
word_freqs = collections.Counter()
num_recs = 0
# DATA_DIR='/Users/gowthamkannan/Desktop/Assignments/Computational_Linguistics/Assignment_2/pos_tagger/data'
# DATA_FILE='dialogues.txt'
ftrain = open(os.path.join(current_dir, file_name), 'rb')
label_list=[]
for line in ftrain:
    words=[]
#     sentence=line.strip().split('\t')[0]
#     label=line.strip().split('\t')[1]
    sentence=line.strip().split(b'DELIMETER')[0]
    label=line.strip().split(b'DELIMETER')[1]
    
#     print(label,sentence)
    words = nltk.word_tokenize(sentence.decode("ascii", "ignore").lower())
    label=label.decode("ascii","ignore").lower()
    label_list.append(label)
#     for item in words_temp:
#         if item not in stopwords_list:
#             words.append(item)
    if len(words) > max_len:
        max_len = len(words)
    for word in words:
        word_freqs[word] += 1
    num_recs += 1
# print(word_freqs)
label_list=list(set(label_list))
nos_classes=len(label_list)
# print(len(label_list))
ftrain.close()
MAX_FEATURES=5000
MAX_SENTENCE_LENGTH=500


In [6]:
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word2index = {x[0]: i+2 for i, x in
enumerate(word_freqs.most_common(MAX_FEATURES))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

In [7]:
X = np.empty((num_recs, ), dtype=list)
nos_classes=2
y = np.zeros((num_recs, nos_classes))
i = 0
f_train = open(os.path.join(current_dir, file_name), 'rb')
for line in f_train:
#     label,sentence=line.strip().split('\t')
    words=[]
    sentence=line.strip().split(b'DELIMETER')[0]
    label=line.strip().split(b'DELIMETER')[1]
    words = nltk.word_tokenize(sentence.decode("ascii", "ignore").lower())
    label=label.decode("ascii","ignore").lower()
    label_list.append(label)
#     for item in words_temp:
#         if item not in stopwords_list:
#             words.append(item)
    seqs=[]
    l=[0]*nos_classes
    for word in words:
        if word in word2index.keys():
            seqs.append(word2index[word])
        else:
            seqs.append(word2index['UNK'])
    X[i]=seqs
    l[label_list.index(label)]=1
    y[i]=l
    i+=1
f_train.close()
print(MAX_SENTENCE_LENGTH)

X=sequence.pad_sequences(X,maxlen=MAX_SENTENCE_LENGTH)

500


In [8]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2)
print(ytrain.shape)

(40000, 2)


In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=MAX_SENTENCE_LENGTH))
model.add(Conv1D(filters=100, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=100, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=32, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160064    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 100)          32100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 100)          100100    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 20, 100)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 20, 32)            32032     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 32)             0         
__________

In [13]:
model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=3, batch_size=75, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 40000 samples, validate on 10000 samples
Epoch 1/3
 - 333s - loss: 0.1958 - acc: 0.9240 - val_loss: 0.2203 - val_acc: 0.9111
Epoch 2/3
 - 307s - loss: 0.1458 - acc: 0.9467 - val_loss: 0.2477 - val_acc: 0.9030
Epoch 3/3
 - 309s - loss: 0.1017 - acc: 0.9654 - val_loss: 0.2691 - val_acc: 0.9047


NameError: name 'X_test' is not defined